## Exploratory Analysis using Jupyter Notebook
For further reading, we recommend: 
- [the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#getting)  for information about using DataFrames
- [this blog post](https://towardsdatascience.com/introduction-to-data-visualization-in-python-89a54c97fbed) for a jumpstart into visualizations
- [the matplotlib documentation](https://matplotlib.org/users/pyplot_tutorial.html) for more info about visualizations

In [1]:
import pandas as pd

#### Loading data from our GPC bucket

In [2]:
import s3fs
s3 = s3fs.S3FileSystem(anon=True)
s3.ls('twde-datalab/raw')

s3.get('twde-datalab/raw/quito_stores_sample2016-2017.csv', 
       '../data/quito_stores_sample2016-2017.csv')

In [33]:
s3.get('twde-datalab/raw/items.csv', 
       '../data/items.csv')

In [59]:
train = pd.read_csv('../data/quito_stores_sample2016-2017.csv')
items = pd.read_csv('../data/items.csv')

In [35]:
train.head()


,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,cluster
0,88211471,2016-08-16,44,103520,7.0,True,Quito,Pichincha,5
1,88211472,2016-08-16,44,103665,7.0,False,Quito,Pichincha,5
2,88211473,2016-08-16,44,105574,13.0,False,Quito,Pichincha,5
3,88211474,2016-08-16,44,105575,18.0,False,Quito,Pichincha,5
4,88211475,2016-08-16,44,105577,8.0,False,Quito,Pichincha,5


In [36]:
items.head()


,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


#### With just this glimpse, you can start to fill out your list of assumptions, hypotheses, and questions. Some of mine are:
- Question: What is the span of dates we are provided?
- Question: How many distinct store_nbr values are there?
- Question: How many distinct item_nbr values are there?
- Hypothesis: unit_sales are always positive
- Hypothesis: onpromotion is always either True or False
- Hypothesis: city and state are always going to be Quito and Pichincha
- Hypothesis: cluster is always 5
- Question: What does cluster mean and is it important to know?
- Question: How many records does the data contain?
- Question: What other data files are available?

In [60]:
train_m = train.merge(items, on='item_nbr', how="left")

In [100]:
train_m.date.resample('W')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [95]:
summary = train_m.groupby(by=['store_nbr', 'date.weekday_name', 'family']).sum()

KeyError: 'date.weekday_name'

In [94]:
summary.sort_values(by=['store_nbr', 'unit_sales'], ascending=False).groupby(by='store_nbr').head(3)

id      item_nbr   unit_sales  onpromotion  \
store_nbr family                                                              
49        GROCERY I  34049120095022  331521041936 3,215,950.00    24,863.00   
          PRODUCE     9232794332272  142863908481 3,115,855.76    14,959.00   
          BEVERAGES  16314321814297  219428660365 2,931,083.00    13,659.00   
48        GROCERY I  32902410828349  317636708862 3,267,712.00    25,999.00   
          BEVERAGES  15510044965002  208751888462 1,977,741.00    13,736.00   
          CLEANING   11276002743493   93645308111   843,321.00    10,865.00   
47        GROCERY I  35187333807491  342030888161 4,078,448.00    27,503.00   
          BEVERAGES  16951368915716  228142712438 3,399,114.00    14,138.00   
          PRODUCE     8300490324545  127629541004 2,304,265.39    13,522.00   
46        GROCERY I  34178141644164  331886846909 3,839,225.00    27,020.00   
          BEVERAGES  16200723070869  218217278927 2,440,860.00    13,693.00   
          PRODUCE     8245552272970  127375684352 1,148,263.76    13,416.00   
45        GROCERY I  35601112105009  345928784273 4,303,133.00    26,866.00   
          BEVERAGES  17264883572235  232208302406 3,601,157.00    14,745.00   
          PRODUCE     8470269895659  130636643835 2,487,242.68    13,827.00   
44        GROCERY I  35405506547717  345777738512 4,052,733.00    27,870.00   
          BEVERAGES  16874926873003  226503148190 3,944,124.00    14,407.00   
          PRODUCE     8824032456525  136376903894 3,738,762.58    14,628.00   

                     cluster      class  perishable  
store_nbr family                                     
49        GROCERY I  3511398  332808385           0  
          PRODUCE     952897  174748838       86627  
          BEVERAGES  1682780  173313747           0  
48        GROCERY I  4316424  321284305           0  
          BEVERAGES  2035390  164771063           0  
          CLEANING   1480584  320232276           0  
47        GROCERY I  4618208  343937670           0  
          BEVERAGES  2224978  180073629           0  
          PRODUCE    1090236  157101808       77874  
46        GROCERY I  4486986  334058924           0  
          BEVERAGES  2126320  172112741           0  
          PRODUCE    1084090  156198946       77435  
45        GROCERY I  3672691  348115897           0  
          BEVERAGES  1780702  183417282           0  
          PRODUCE     873807  160236470       79437  
44        GROCERY I  1660510  346244422           0  
          BEVERAGES   791475  179326718           0  
          PRODUCE     413705  166901424       82741

In [85]:
help(pd.DataFrame.sort_values)

Help on function sort_values in module pandas.core.frame:

sort_values(self, by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
    Sort by the values along either axis
    
    Parameters
    ----------
            by : str or list of str
                Name or list of names to sort by.
    
                - if `axis` is 0 or `'index'` then `by` may contain index
                  levels and/or column labels
                - if `axis` is 1 or `'columns'` then `by` may contain column
                  levels and/or index labels
    
                .. versionchanged:: 0.23.0
                   Allow specifying index or column level names.
    axis : {0 or 'index', 1 or 'columns'}, default 0
         Axis to be sorted
    ascending : bool or list of bool, default True
         Sort ascending vs. descending. Specify list for multiple sort
         orders.  If this is a list of bools, must match the length of
         the by.
    inplace : bool, default Fa

### Here's some examples of how to address those first questions

In [7]:
# Access an entire dataframe column like you would
# the value in a python dictionary:
# (The returned object has similar pandas built-in 
# functions, like 'head' and 'max')
data =train
print(data['date'].min())
print(data['date'].max())

2016-08-16
2017-08-15


In [8]:
# Dataframe columns also have a 'unique' method,
# which can answer several of our questions from above
data['store_nbr'].unique()

array([44, 45, 46, 47, 48, 49])

In [9]:
print(data['item_nbr'].unique())
print("There are too many item numbers to display, so let's just count them for now:")
print("\n{} different item_nbr values in our data"
          .format(len(data['item_nbr'].unique())))

[ 103520  103665  105574 ... 2011468 2011448 2123839]
There are too many item numbers to display, so let's just count them for now:

3717 different item_nbr values in our data


#### It might be helpful to know the 'shape' of our data. We could count by hand (for now) the columns, but how many rows do we have altogether?

In [10]:
print(data.shape)
print("There are {} rows and {} columns in our data".format(data.shape[0], data.shape[1]))

(5877318, 9)
There are 5877318 rows and 9 columns in our data


#### Moving along to answer our intial questions... Let's have a look at unit_sales. Keep in mind that unit sales is the variable we want to predict with our science.

Each row in our data is essentially telling us a `unit_sales` number for a given `item_nbr` at a given `store_nbr` on a given `date`. That is, "how many of an item was sold at a store on a day".

In [14]:
len(data[data['unit_sales']< 0])

477

In [13]:
len(data[data['unit_sales']==0])

0

In [18]:
returns = data[data['unit_sales']< 0]
returns['store_nbr'].unique()


array([46, 45, 44, 49, 48, 47])

In [17]:
min(returns['unit_sales'])

-290.0

In [19]:
data['cluster'].unique()

array([ 5, 11, 14])

In [21]:
returns['onpromotion'].unique()

array([False,  True])

In [22]:
returns['date'].unique()

array(['2016-08-16', '2016-08-19', '2016-08-20', '2016-08-21',
       '2016-08-23', '2016-08-26', '2016-08-29', '2016-08-30',
       '2016-08-31', '2016-09-02', '2016-09-04', '2016-09-05',
       '2016-09-07', '2016-09-08', '2016-09-09', '2016-09-10',
       '2016-09-12', '2016-09-13', '2016-09-14', '2016-09-15',
       '2016-09-16', '2016-09-20', '2016-09-22', '2016-09-26',
       '2016-09-27', '2016-09-28', '2016-09-29', '2016-10-02',
       '2016-10-03', '2016-10-04', '2016-10-05', '2016-10-06',
       '2016-10-07', '2016-10-09', '2016-10-10', '2016-10-12',
       '2016-10-13', '2016-10-14', '2016-10-15', '2016-10-17',
       '2016-10-19', '2016-10-20', '2016-10-22', '2016-10-24',
       '2016-10-25', '2016-10-27', '2016-10-30', '2016-10-31',
       '2016-11-01', '2016-11-02', '2016-11-04', '2016-11-05',
       '2016-11-06', '2016-11-07', '2016-11-08', '2016-11-09',
       '2016-11-10', '2016-11-11', '2016-11-13', '2016-11-15',
       '2016-11-17', '2016-11-18', '2016-11-19', '2016-

In [24]:
cluster14 = data[data['cluster']==14]
cluster14['unit_sales'].sum()

35445654.047

In [26]:
cluster11 = data[data['cluster']==11]
cluster11['unit_sales'].sum()

28937304.692000005

In [27]:
cluster5 = data[data['cluster']==5]
cluster5['unit_sales'].sum()

17013505.713

In [28]:
cluster14 = data[data['cluster']==14]
cluster14['date'].min()

'2016-08-16'

In [29]:
cluster14 = data[data['cluster']==14]
cluster14['date'].max()

'2017-08-15'

In [31]:
cluster = data[data['cluster']==11]
print("min date {} max date {}", cluster['date'].min(), cluster['date'].max())

min date {} max date {} 2016-08-16 2017-08-15


In [32]:
cluster = data[data['cluster']==5]
print("min date {} max date {}", cluster['date'].min(), cluster['date'].max())

min date {} max date {} 2016-08-16 2017-08-15
